In [2]:
#import library yang dibutuhkan
import pandas as pd
import numpy as np
import re
import string
import nltk
import warnings
warnings.filterwarnings("ignore")
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [3]:
#input data yang ingin dicleansing
def load_data():
    data = pd.read_csv('data.csv', encoding='latin-1')
    return data

In [4]:
abusive_df = load_data()
abusive_df

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13164,USER jangan asal ngomong ndasmu. congor lu yg ...,1,1,1,0,0,0,1,0,0,1,0,0
13165,USER Kasur mana enak kunyuk',0,1,0,0,0,0,0,0,0,0,0,0
13166,USER Hati hati bisu :( .g\n\nlagi bosan huft \...,0,0,0,0,0,0,0,0,0,0,0,0
13167,USER USER USER USER Bom yang real mudah terdet...,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
#memilih kolom yang ingin dipakai
df=pd.DataFrame(abusive_df[['Tweet']])
df

,Tweet
0,- disaat semua cowok berusaha melacak perhatia...
1,RT USER: USER siapa yang telat ngasih tau elu?...
2,"41. Kadang aku berfikir, kenapa aku tetap perc..."
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...
4,USER USER Kaum cebong kapir udah keliatan dong...
...,...
13164,USER jangan asal ngomong ndasmu. congor lu yg ...
13165,USER Kasur mana enak kunyuk'
13166,USER Hati hati bisu :( .g\n\nlagi bosan huft \...
13167,USER USER USER USER Bom yang real mudah terdet...


In [6]:
#check dataframe size (baris, kolom)
df.shape

(13169, 1)

In [7]:
#check tipe data di setiap kolom
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13169 entries, 0 to 13168
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   13169 non-null  object
dtypes: object(1)
memory usage: 103.0+ KB


In [8]:
#cleansing dari missing value
df.dropna()

,Tweet
0,- disaat semua cowok berusaha melacak perhatia...
1,RT USER: USER siapa yang telat ngasih tau elu?...
2,"41. Kadang aku berfikir, kenapa aku tetap perc..."
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...
4,USER USER Kaum cebong kapir udah keliatan dong...
...,...
13164,USER jangan asal ngomong ndasmu. congor lu yg ...
13165,USER Kasur mana enak kunyuk'
13166,USER Hati hati bisu :( .g\n\nlagi bosan huft \...
13167,USER USER USER USER Bom yang real mudah terdet...


In [9]:
#case folding (mengubah ke dalam lower case)
df['Case Folding'] = df['Tweet'].str.lower()
df.head()

,Tweet,Case Folding
0,- disaat semua cowok berusaha melacak perhatia...,- disaat semua cowok berusaha melacak perhatia...
1,RT USER: USER siapa yang telat ngasih tau elu?...,rt user: user siapa yang telat ngasih tau elu?...
2,"41. Kadang aku berfikir, kenapa aku tetap perc...","41. kadang aku berfikir, kenapa aku tetap perc..."
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,user user aku itu aku\n\nku tau matamu sipit t...
4,USER USER Kaum cebong kapir udah keliatan dong...,user user kaum cebong kapir udah keliatan dong...


In [10]:
#Remve special character
def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = str(text).replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"").replace('\n',"").replace('\ud83d',"").replace('\ude4f',"").replace('\ud83e',"").replace('\udd23',"").replace('\ude2d',"").replace('\udc40',"")
    
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    text = re.sub(r'^rt[\s]+', '', text)
    
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

df['Data Cleansing']= df['Case Folding'].apply(remove_tweet_special)
df.head()

,Tweet,Case Folding,Data Cleansing
0,- disaat semua cowok berusaha melacak perhatia...,- disaat semua cowok berusaha melacak perhatia...,- disaat semua cowok berusaha melacak perhatia...
1,RT USER: USER siapa yang telat ngasih tau elu?...,rt user: user siapa yang telat ngasih tau elu?...,user: user siapa yang telat ngasih tau elu?eda...
2,"41. Kadang aku berfikir, kenapa aku tetap perc...","41. kadang aku berfikir, kenapa aku tetap perc...","41. kadang aku berfikir, kenapa aku tetap perc..."
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,user user aku itu aku\n\nku tau matamu sipit t...,user user aku itu aku ku tau matamu sipit tapi...
4,USER USER Kaum cebong kapir udah keliatan dong...,user user kaum cebong kapir udah keliatan dong...,user user kaum cebong kapir udah keliatan dong...


In [11]:
#remove number
def remove_number(text):
    return re.sub(r"[.,:;+!\-_<^/=?\"'\(\)\d\*]", " ", text)
df['Data Cleansing']= df['Data Cleansing'].apply(remove_number)

#Remove Punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))
df['Data Cleansing']= df['Data Cleansing'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()
df['Data Cleansing']=df['Data Cleansing'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)
df['Data Cleansing']= df['Data Cleansing'].apply(remove_whitespace_multiple)

#remove user
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt    
df['Data Cleansing'] = np.vectorize(remove_pattern)(df['Data Cleansing'], "@[\w]*")

# remove single char
def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)
df['Data Cleansing']= df['Data Cleansing'].apply(remove_single_char)

df.head()

,Tweet,Case Folding,Data Cleansing
0,- disaat semua cowok berusaha melacak perhatia...,- disaat semua cowok berusaha melacak perhatia...,disaat semua cowok berusaha melacak perhatian ...
1,RT USER: USER siapa yang telat ngasih tau elu?...,rt user: user siapa yang telat ngasih tau elu?...,user user siapa yang telat ngasih tau elu edan...
2,"41. Kadang aku berfikir, kenapa aku tetap perc...","41. kadang aku berfikir, kenapa aku tetap perc...",kadang aku berfikir kenapa aku tetap percaya p...
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,user user aku itu aku\n\nku tau matamu sipit t...,user user aku itu aku ku tau matamu sipit tapi...
4,USER USER Kaum cebong kapir udah keliatan dong...,user user kaum cebong kapir udah keliatan dong...,user user kaum cebong kapir udah keliatan dong...


In [12]:
#remove duplicate
df = df.drop_duplicates(subset=['Data Cleansing'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12926 entries, 0 to 13168
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Tweet           12926 non-null  object
 1   Case Folding    12926 non-null  object
 2   Data Cleansing  12926 non-null  object
dtypes: object(3)
memory usage: 403.9+ KB


In [13]:
#Tokenization
def tokenization(text):
    text = re.split('\W+', text)
    return text
df['Tokenization']= df['Data Cleansing'].apply(tokenization)


In [14]:
#Normalisasi

from nltk.corpus import stopwords

#stopword indonesia
stopword = stopwords.words('indonesian')
stopword.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

stopword = set(stopword)

def stopwords_removal(text):
    text= [word for word in text if word not in stopword]
    return text

df['Stopwords']=df['Tokenization'].apply(stopwords_removal)
df.head()

,Tweet,Case Folding,Data Cleansing,Tokenization,Stopwords
0,- disaat semua cowok berusaha melacak perhatia...,- disaat semua cowok berusaha melacak perhatia...,disaat semua cowok berusaha melacak perhatian ...,"[disaat, semua, cowok, berusaha, melacak, perh...","[disaat, cowok, berusaha, melacak, perhatian, ..."
1,RT USER: USER siapa yang telat ngasih tau elu?...,rt user: user siapa yang telat ngasih tau elu?...,user user siapa yang telat ngasih tau elu edan...,"[user, user, siapa, yang, telat, ngasih, tau, ...","[user, user, telat, ngasih, elu, edan, sarap, ..."
2,"41. Kadang aku berfikir, kenapa aku tetap perc...","41. kadang aku berfikir, kenapa aku tetap perc...",kadang aku berfikir kenapa aku tetap percaya p...,"[kadang, aku, berfikir, kenapa, aku, tetap, pe...","[kadang, berfikir, percaya, tuhan, jatuh, berk..."
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,user user aku itu aku\n\nku tau matamu sipit t...,user user aku itu aku ku tau matamu sipit tapi...,"[user, user, aku, itu, aku, ku, tau, matamu, s...","[user, user, ku, matamu, sipit, diliat]"
4,USER USER Kaum cebong kapir udah keliatan dong...,user user kaum cebong kapir udah keliatan dong...,user user kaum cebong kapir udah keliatan dong...,"[user, user, kaum, cebong, kapir, udah, keliat...","[user, user, kaum, cebong, kapir, udah, keliat..."


In [45]:
import swifter

import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['Stopwords']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['Stemmer'] = df['Stopwords'].apply(get_stemmed_term)
print(df['Stemmer'])

25287
------------------------
disaat : saat
cowok : cowok
berusaha : usaha
melacak : lacak
perhatian : perhati
gue : gue
loe : loe
lantas : lantas
remehkan : remeh
kasih : kasih
khusus : khusus
elo : elo
basic : basic
bego : bego
user : user
telat : telat
ngasih : ngasih
elu : elu
edan : edan
sarap : sarap
bergaul : gaul
cigax : cigax
jifla : jifla
calis : cal
noh : noh
licew : licew
kadang : kadang
berfikir : berfikir
percaya : percaya
tuhan : tuhan
jatuh : jatuh
berkali : kali
kali : kali
ninggalkan : ninggalkan
orangtuaku : orangtuaku
berencana : rencana
berpisah : pisah
kakakku : kakak
memilih : pilih
kristen : kristen
anak : anak
ter : ter
ku : ku
matamu : mata
sipit : sipit
diliat : liat
kaum : kaum
cebong : cebong
kapir : kapir
udah : udah
keliatan : liat
dongoknya : dongok
dongok : dongok
hahahah : hahahah
bani : bani
taplak : taplak
dkk : dkk
xf : xf
fx : fx
 : 
deklarasi : deklarasi
pilkada : pilkada
aman : aman
anti : anti
hoax : hoax
warga : warga
dukuh : dukuh
sari : sari

klau : klau
diendahkan : diendahkan
exb : exb
la : la
hm : hm
hmm : hmm
exa : exa
diam : diam
bisu : bisu
coba : coba
rumput : rumput
bergoyang : goyang
xabxe : xabxe
xaaxe : xaaxe
xaa : xaa
membisikkan : bisik
was : was
kepadamu : kepada
mintalah : minta
perlindungan : lindung
qs : qs
axe : axe
raf : raf
fushilat : fushilat
klw : klw
ente : ente
atheis : atheis
ngapain : ngapain
sibuk : sibuk
bangsat : bangsat
selamat : selamat
kartini : kartini
kamar : kamar
mandi : mandi
laki : laki
biarkan : biar
cocot : cocot
ngajarin : ngajarin
solat : solat
ajah : ajah
memecah : pecah
belah : belah
lo : lo
ngomong : ngomong
kaga : kaga
bener : bener
bangke : bangke
mempercayai : percaya
negara : negara
disuatu : suatu
dijajah : jajah
kerana : rana
kes : kes
pembubaran : bubar
sistem : sistem
beraja : beraja
dinegara : negara
menjadikan : jadi
republik : republik
komunis : komunis
penjaja : jaja
kebijakan : bijak
diusulkan : usul
yo : yo
opo : opo
seumpama : umpama
cina : cina
usir : usir
paksa :

ckakakakak : ckakakakak
menekan : tekan
impossible : impossible
dilelang : lelang
murah : murah
ntb : ntb
contoh : contoh
menang : menang
caya : caya
buaya : buaya
mu : mu
bwang : bwang
menyatu : satu
melawan : lawan
mega : mega
peristiwa : peristiwa
terulang : ulang
dimana : mana
incumbent : incumbent
dikalahkan : kalah
mutlak : mutlak
jk : jk
jkw : jkw
mk : mk
poros : poros
kayak : kayak
gatot : gatot
gaduh : gaduh
gentleman : gentleman
bersaing : saing
presidens : presidens
bebal : bebal
populasi : populasi
terbesar : besar
dunia : dunia
musuh : musuh
barat : barat
menghambat : hambat
kepentingan : penting
obviously : obviously
umuh : umuh
persib : persib
kunyuk : kunyuk
film : film
pilih : pilih
nggak : nggak
sholatin : sholatin
dijamin : jamin
laris : laris
banu : banu
yahudi : yahudi
syriac : syriac
perbedaan : beda
tetep : tetep
eloah : eloah
elohim : elohim
yahwe : yahwe
alah : alah
muka : muka
merah : merah
dpt : dpt
cuaca : cuaca
ni : ni
panas : panas
pemberitaan : berita
sar

kebusukab : kebusukab
berkah : berkah
cuci : cuci
wkwkwkwk : wkwkwkwk
bikan : bik
ego : ego
klhk : klhk
upayakan : upaya
pemerataan : perata
tora : tora
kawasan : kawasan
hutan : hutan
tribun : tribun
medan : medan
mahfud : mahfud
md : md
berpaling : paling
swt : swt
pemberang : berang
ajak : ajak
kotoran : kotor
somplak : somplak
terkuak : kuak
strategi : strategi
kampanye : kampanye
pemenangan : menang
baduy : baduy
memperingati : ingat
seba : seba
pendopo : pendopo
pemkab : pemkab
lebak : lebak
dilanjutkan : lanjut
bertemu : temu
banten : banten
wahidin : wahidin
halim : halim
sabtu : sabtu
lembaga : lembaga
bubar : bubar
sentuh : sentuh
kekuasaan : kuasa
giliran : gilir
plg : plg
ngotot : ngotot
bubarkan : bubar
mnyudutkan : mnyudutkan
trtentu : trtentu
bahaya : bahaya
rocky : rocky
cerdik : cerdik
nyusun : nyusun
kalimat : kalimat
dgunakan : dgunakan
inti : inti
pngertian : pngertian
sbenarnya : sbenarnya
ayolah : ayo
move : move
salalu : salalu
urus : urus
rp : rp
ngak : ngak
did

ahhh : ahhh
perasaanmu : asa
menghibur : hibur
tol : tol
germo : germo
resah : resah
penuh : penuh
drama : drama
main : main
ftv : ftv
judulnya : judul
darah : darah
fernando : fernando
valley : valley
bunuh : bunuh
jahudi : jahudi
jerman : jerman
ibadah : ibadah
papamintaketuafraksi : papamintaketuafraksi
setya : setya
novanto : novanto
pecat : pecat
biawak : biawak
meroket : roket
sekop : sekop
karbitan : karbitan
munafik : munafik
blabla : blabla
emoh : emoh
agitasi : agitasi
simpatisan : simpatisan
berharap : harap
skema : skema
pemilu : milu
bwt : bwt
sj : sj
karung : karung
mgkn : mgkn
mekanisme : mekanisme
ok : ok
ditentukan : tentu
berprestasi : prestasi
vital : vital
inter : inter
bek : bek
kanan : kanan
pembeda : beda
maicon : maicon
ambrosio : ambrosio
kedatangan : datang
cancelo : cancelo
bagusnya : bagus
hmmmm : hmmmm
kesimpulannya : simpul
geblek : geblek
ausilio : ausilio
tudingan : tuding
dugaan : duga
dorong : dorong
kejaksaan : jaksa
usut : usut
himbau : himbau
menciu

...

lamany : lamany
imej : imej
diaspora : diaspora
sehina : hina
longkang : longkang
sontot : sontot
sofil : sofil
luarr : luarr
staff : staff
monoton : monoton
atu : atu
melas : melas
sjxe : sjxe
shalatjumat : shalatjumat
disela : sela
nyelempit : nyelempit
bernaung : naung
wkkw : wkkw
rebu : rebu
asuh : asuh
kenetralannya : netral
dilacurkan : lacur
personil : personil
provost : provost
bripka : bripka
kopdit : kopdit
gentiaras : gentiaras
toel : toel
anjiiiing : anjiiiing
cengk : cengk
umpetin : umpetin
sudoku : sudoku
keruan : keruan
tertantang : tantang
padet : padet
slusap : slusap
slusup : slusup
trsusun : trsusun
mnyematkn : mnyematkn
kberagamn : kberagamn
dstiap : dstiap
hrufny : hrufny
saman : saman
sasirangan : sasirangan
banjarmasin : banjarmasin
brasal : brasal
barong : barong
boongan : boongan
ochuxf : ochuxf
tekxe : tekxe
beresiko : resiko
tkng : tkng
ngalus : ngalus
eet : eet
ditetapkn : ditetapkn
didlm : didlm
kawasn : kawasn
dirubh : dirubh
obati : obat
menggerogoti : ge

wall : wall
diplomasi : diplomasi
berilmu : beril
bomxe : bomxe
ntn : ntn
radha : radha
dat : dat
muah : muah
makamya : makamya
sissy : sissy
bajay : bajay
asuan : asu
lepeh : lepeh
sukuran : sukuran
mumetxf : mumetxf
cermah : cermah
wasatiyah : wasatiyah
kasihannya : kasihan
yf : yf
yok : yok
maijen : maijen
xxi : xxi
distudio : studio
dilayarnya : layar
sihabxf : sihabxf
tipiwan : tipiwan
tipi : tipi
lynde : lynde
sitepu : sitepu
jogging : jogging
dipersilahkan : silah
menghampaskan : menghampaskan
sofa : sofa
wiraxe : wiraxe
memposisikan : posisi
senyama : senyama
bocahemak : bocahemak
gapain : gapain
mengexpose : mengexpose
zionisme : zionisme
expose : expose
senangnya : senang
mengadudombakan : mengadudombakan
gegabah : gegabah
mendasarkan : dasar
foreign : foreign
inflow : inflow
banyk : banyk
uuuu : uuuu
mtk : mtk
atletis : atlet
makian : maki
shopping : shopping
syarikat : syarikat
bubarkangmbi : bubarkangmbi
copotkapoldajabar : copotkapoldajabar
selamatkanulama : selamatkanula

sebwah : sebwah
mentol : mentol
kibul : kibul
disekolahin : disekolahin
pintet : pintet
prokorupt : prokorupt
perjalananmu : jalan
ditelaga : telaga
telaga : telaga
menjerit : jerit
sangkalan : sangkal
senopati : senopati
andalannya : andal
gakdeng : gakdeng
bangcat : bangcat
sedeket : sedeket
oped : oped
terperuk : peruk
athirah : athirah
meneernya : meneernya
hadeuuuh : hadeuuuh
mengimpor : impor
sentil : sentil
umpama : umpama
bkxc : bkxc
lantaklah : lantak
omdo : omdo
membuk : buk
pangandaran : pangandaran
elegi : elegi
figura : figura
renata : renata
betoi : betoi
maqbul : maqbul
khaleel : khaleel
pelita : pelita
ditinggalkan : tinggal
komenannya : komenannya
ampera : ampera
liput : liput
yadong : yadong
hellow : hellow
opsisi : opsisi
any : any
ijma : ijma
bismillaah : bismillaah
lubuk : lubuk
berikutnye : berikutnye
diberkahi : kah
sbgn : sbgn
berpikirnya : pikir
diributin : diributin
trsalurkan : trsalurkan
fansnya : fansnya
ramaikan : ramai
coment : coment
expert : expert
awan

sajah : sajah
penghukuman : hukum
nuhan : nuhan
sakaw : sakaw
sdn : sdn
tikep : tikep
sman : sman
oleo : oleo
mtmu : mtmu
skipxf : skipxf
memberlakukan : laku
perjudian : judi
lomba : lomba
menghias : hias
sekelasku : kelas
ambis : ambis
hadiahnya : hadiah
xbb : xbb
kedoyanan : doyan
uwel : uwel
gkgkgk : gkgkgk
omelan : omel
ngunyah : ngunyah
kuntinya : kuntinya
melencengkan : lenceng
didirikan : diri
ekstra : ekstra
sambunvan : sambunvan
kekehan : kekeh
ocha : ocha
bergegas : gegas
milly : milly
eds : eds
bermigrasi : migrasi
perfilman : film
kawakan : kawak
deddy : deddy
sutomo : sutomo
fadlizone : fadlizone
ketulian : tuli
kesulitan : sulit
wkkwkwkw : wkkwkwkw
memelukmu : peluk
membiarkanmu : biar
bersimpuh : simpuh
buktik : buktik
meluat : luat
kenaxe : kenaxe
meledakan : ledak
tewaskan : tewas
tersebar : sebar
muallaf : muallaf
menginspirasi : inspirasi
pocong : pocong
mck : mck
hyun : hyun
berlakon : lakon
menitik : titik
cikupa : cikupa
smpah : smpah
kurap : kurap
seorganisasi :

brbeda : brbeda
haya : haya
slesai : slesai
mmperingati : mmperingati
brtemu : brtemu
herlambang : herlambang
kesatu : satu
simpatik : simpatik
alaaaah : alaaaah
poligamy : poligamy
radenmas : radenmas
mskpun : mskpun
inflas : inflas
pkrannya : pkrannya
kekuasan : kuas
dipublikasikan : publikasi
kamuu : kamuu
imutan : imutan
semangatt : semangatt
mahxf : mahxf
namax : namax
jejeritan : jejeritan
mengenaliku : kenal
marni : marni
bngsat : bngsat
trmsuk : trmsuk
pngikut : pngikut
mnyimpang : mnyimpang
ngetuit : ngetuit
petromax : petromax
beresss : beresss
looo : looo
nafsunya : nafsu
kelicikan : kelici
masadepaan : masadepaan
cerai : cerai
nnyatanya : nnyatanya
artidjo : artidjo
ngebatalin : ngebatalin
lagiiiiiiii : lagiiiiiiii
gegarotan : gegarotan
pencapain : pencapain
ngelebarin : ngelebarin
dipakenya : dipakenya
sengakak : sengakak
ngakaknya : ngakaknya
kenap : kenap
ditelan : tel
nelpon : nelpon
debora : debora
rasakeenn : rasakeenn
falsu : falsu
kehancurannya : hancur
beraliran : 

In [15]:
df.head()

,Tweet,Case Folding,Data Cleansing,Tokenization,Stopwords
0,- disaat semua cowok berusaha melacak perhatia...,- disaat semua cowok berusaha melacak perhatia...,disaat semua cowok berusaha melacak perhatian ...,"[disaat, semua, cowok, berusaha, melacak, perh...","[disaat, cowok, berusaha, melacak, perhatian, ..."
1,RT USER: USER siapa yang telat ngasih tau elu?...,rt user: user siapa yang telat ngasih tau elu?...,user user siapa yang telat ngasih tau elu edan...,"[user, user, siapa, yang, telat, ngasih, tau, ...","[user, user, telat, ngasih, elu, edan, sarap, ..."
2,"41. Kadang aku berfikir, kenapa aku tetap perc...","41. kadang aku berfikir, kenapa aku tetap perc...",kadang aku berfikir kenapa aku tetap percaya p...,"[kadang, aku, berfikir, kenapa, aku, tetap, pe...","[kadang, berfikir, percaya, tuhan, jatuh, berk..."
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,user user aku itu aku\n\nku tau matamu sipit t...,user user aku itu aku ku tau matamu sipit tapi...,"[user, user, aku, itu, aku, ku, tau, matamu, s...","[user, user, ku, matamu, sipit, diliat]"
4,USER USER Kaum cebong kapir udah keliatan dong...,user user kaum cebong kapir udah keliatan dong...,user user kaum cebong kapir udah keliatan dong...,"[user, user, kaum, cebong, kapir, udah, keliat...","[user, user, kaum, cebong, kapir, udah, keliat..."


In [16]:
#Penggabungan kata
def fit_stopwords(text):
    text= np.array(text)
    text= ' '.join(text)
    return text

df['Text']= df['Stemmer'].apply(lambda x: fit_stopwords(x))
df.head()

KeyError: 'Stemmer'

In [40]:
df.to_csv("Hasil_Preprocesing.csv")